# Things to consider for continue

1. simulation with rankig over time


In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [2]:
# dt = datetime.strptime('2020-04-01', '%Y-%m-%d')
dt = datetime.today()  # .date()
str(dt)

'2024-04-10 08:11:17.969872'

In [3]:
def read_fights():
    path = r"..\..\data\all_fights_new.csv"
    df = pd.read_csv(path, parse_dates=True, low_memory=False)[
        [
            "Date",
            "fighter1",
            "fighter2",
            "fighter1_result",
            "fighter2_result",
            "BELT",
            "BOUNES",
            "Weight",
            "TIME_FORMAT",
            "Method",
            "Method_detail",
            "Judge1_score",
            "Judge2_score",
            "Judge3_score",
            "ROUND",
            "TIME",
        ]
    ]
    df = df[df["Date"] != "Date"]
    df.BOUNES = (df.BOUNES.str.split("/")).str[-1].str.split(".png").str[0]
    df.BELT = (df.BELT.str.split("/")).str[-1].str.split(".png").str[0]
    df.Weight = df.Weight.str.strip("Bout")
    df.Date = pd.to_datetime(df.Date.astype(str))
    df.TIME_FORMAT = df.TIME_FORMAT.str.split(" ").str[0]  # .astype('int')
    df = df[df["Date"] < str(dt)]
    return df


df = read_fights()

In [4]:
def weight(w):
    if "Women's" in w:
        if "Strawweight" in w:
            new = "Women's Strawweight"
        elif "Flyweight" in w:
            new = "Women'sFlyweight"
        elif "antamweight" in w:
            new = "Women's Bantamweight"
        elif "Featherweight" in w:
            new = "Women's Featherweight"
    elif "Flyweight" in w:
        new = "Flyweight"
    elif "antamweight" in w:
        new = "Bantamweight"
    elif "Featherweight" in w:
        new = "Featherweight"
    elif "Lightweight" in w:
        new = "Lightweight"
    elif "Welterweight" in w:
        new = "Welterweight"
    elif "Middleweight" in w:
        new = "Middleweight"
    elif "Light Heavyweight" in w:
        new = "Light Heavyweight"
    elif "Heavyweight" in w:
        new = "Heavyweight"
    else:
        new = "other"

    return new


df["Weight"] = df.apply(lambda x: weight(x.Weight), axis=1)

In [5]:
def fighter_perfomance(n):
    """
    n: number of years for performance rate
    """
    df_1 = (
        df[(df["Date"] >= dt - timedelta(weeks=52 * n))]
        .groupby(["fighter1", "fighter1_result", "Weight"])["TIME_FORMAT"]
        .count()
        .unstack(level=-2)
        .fillna(0)
    )
    df_1.index.set_names(["fighter", "weight"], inplace=True)
    df_2 = (
        df[(df["Date"] >= dt - timedelta(weeks=52 * n))]
        .groupby(["fighter2", "fighter2_result", "Weight"])["TIME_FORMAT"]
        .count()
        .unstack(level=-2)
        .fillna(0)
    )
    df_2.index.set_names(["fighter", "weight"], inplace=True)
    df_perf = df_1.add(df_2, fill_value=0)
    df_perf["n_fights"] = df_perf.sum(axis=1)
    df_perf["perf_rat"] = (df_perf["n_fights"] - df_perf["L"]) / df_perf["n_fights"]
    # df_perf.columns = df_perf.columns.values
    df_perf.columns = pd.Index(df_perf.columns.values.astype(str))
    df_perf = df_perf.sort_values(
        by=["perf_rat", "n_fights"], ascending=False
    ).reset_index(
        level=-1
    )  # [df_perf.n_fights>2]
    return df_perf


fighter_perfomance(10).head()

,weight,D,L,NC,W,n_fights,perf_rat
fighter,,,,,,,
Jack Della Maddalena,Welterweight,0.0,0.0,0.0,8.0,8.0,1.0
Jon Jones,Light Heavyweight,0.0,0.0,1.0,7.0,8.0,1.0
Dricus Du Plessis,Middleweight,0.0,0.0,0.0,7.0,7.0,1.0
Ian Machado Garry,Welterweight,0.0,0.0,0.0,7.0,7.0,1.0
Manon Fiorot,Women'sFlyweight,0.0,0.0,0.0,7.0,7.0,1.0


In [6]:
# list(df['Weight'].value_counts().index)

In [7]:
# adjusts the winner and losser position
def win_lose(fighter1_result, fighter1, fighter2):
    if fighter1_result == "W":
        return fighter1, fighter2
    elif fighter1_result == "L":
        return fighter2, fighter1
    else:
        return fighter2, fighter1


df["winner"] = df.apply(
    lambda x: win_lose(x.fighter1_result, x.fighter1, x.fighter2), axis=1
).str[0]
df["losser"] = df.apply(
    lambda x: win_lose(x.fighter1_result, x.fighter1, x.fighter2), axis=1
).str[1]
df = df[
    [
        "Date",
        "winner",
        "losser",
        "BELT",
        "BOUNES",
        "Weight",
        "TIME_FORMAT",
        "Method",
        "Method_detail",
        "Judge1_score",
        "Judge2_score",
        "Judge3_score",
        "ROUND",
        "TIME",
        "fighter1",
        "fighter2",
        "fighter1_result",
        "fighter2_result",
    ]
]

In [8]:
df.head()

,Date,winner,losser,BELT,BOUNES,Weight,TIME_FORMAT,Method,Method_detail,Judge1_score,Judge2_score,Judge3_score,ROUND,TIME,fighter1,fighter2,fighter1_result,fighter2_result
0,2022-10-22,Muhammad Mokaev,Malcolm Gordon,NaN,NaN,Flyweight,3,Submission,Armbar From Back Control,NaN,NaN,NaN,3,4:26,Muhammad Mokaev,Malcolm Gordon,W,L
1,2022-10-22,Armen Petrosyan,AJ Dobson,NaN,NaN,Middleweight,3,Decision - Unanimous,NaN,27 - 30.,27 - 30.,27 - 30.,3,5:00,Armen Petrosyan,AJ Dobson,W,L
2,2022-10-22,Karol Rosa,Lina Lansberg,NaN,NaN,Women's Bantamweight,3,Decision - Majority,NaN,27 - 29.,27 - 29.,28 - 28.,3,5:00,Karol Rosa,Lina Lansberg,W,L
3,2022-12-03,Natan Levy,Genaro Valdez,NaN,NaN,Lightweight,3,Decision - Unanimous,NaN,28 - 29.,28 - 29.,27 - 30.,3,5:00,Natan Levy,Genaro Valdez,W,L
4,2022-12-03,Yazmin Jauregui,Istela Nunes,NaN,NaN,Women's Strawweight,3,KO/TKO,Punches to Head From Guard,NaN,NaN,NaN,2,4:06,Yazmin Jauregui,Istela Nunes,W,L


In [9]:
# adjust Draws
# it will add a line for the draw cases to compansate the win-loose
def draw_adj(df):
    df_d = df[df["fighter1_result"] == "D"].copy()

    df_d["w_trans"] = df_d["winner"]
    df_d["winner"] = df_d["losser"]
    df_d["losser"] = df_d["w_trans"]
    df_d.drop("w_trans", axis=1, inplace=True)
    df = pd.concat([df, df_d], axis=0)
    return df


df = draw_adj(df)

In [12]:
# clean method
def method(m):
    if "Decision" in m:
        new = "Decision"
    elif "KO" in m:
        new = "KO"
    elif "Submission" in m:
        new = "sub"
    else:
        new = "other"
    return new


df["method"] = df.apply(lambda x: method(x.Method), axis=1)

In [13]:
# assign values to method
def val(row):
    if row.fighter1_result == "D":
        new = 0
    elif row.fighter1_result == "NC":
        new = 0
    elif row.method == "Decision":
        new = 5
    else:
        new = 7

    if row.BELT == "belt":
        new = new * 2
    return new


df["value"] = df.apply(lambda x: val(x), axis=1)

In [14]:
# recencey ration
def recency(Date, dt):
    import math

    # ((dt - df['Date']).dt.days/365).apply(math.floor)
    date = math.floor((dt - Date).days / 365)
    if date < 3:
        val = 1
    elif date > 3 & date < 10:
        val = 1 - ((date - 2) / 10)
    else:
        val = 0.1
    return val


df["adj"] = df.apply(lambda x: recency(x.Date, dt), axis=1)
df["adj_val"] = df["adj"] * df["value"]
df.sort_values(by=["Date"], ascending=False, inplace=True)

In [15]:
df.head()

,Date,winner,losser,BELT,BOUNES,Weight,TIME_FORMAT,Method,Method_detail,Judge1_score,...,ROUND,TIME,fighter1,fighter2,fighter1_result,fighter2_result,method,value,adj,adj_val
7591,2024-03-30,Kyle Nelson,Bill Algeo,NaN,NaN,Featherweight,3,KO/TKO,Punches to Head At Distance,NaN,...,1,4:00,Bill Algeo,Kyle Nelson,L,W,KO,7,1.0,7.0
7595,2024-03-30,Manon Fiorot,Erin Blanchfield,NaN,NaN,Women'sFlyweight,5,Decision - Unanimous,NaN,45 - 50.,...,5,5:00,Erin Blanchfield,Manon Fiorot,L,W,Decision,5,1.0,5.0
7589,2024-03-30,Chidi Njokuani,Rhys McKee,NaN,NaN,Welterweight,3,Decision - Split,NaN,29 - 28.,...,3,5:00,Chidi Njokuani,Rhys McKee,W,L,Decision,5,1.0,5.0
7590,2024-03-30,Virna Jandiroba,Loopy Godinez,NaN,NaN,Women's Strawweight,3,Decision - Unanimous,NaN,28 - 29.,...,3,5:00,Virna Jandiroba,Loopy Godinez,W,L,Decision,5,1.0,5.0
7586,2024-03-30,Julio Arce,Herbert Burns,NaN,NaN,Featherweight,3,KO/TKO,Punch to Head At Distance,NaN,...,2,2:00,Julio Arce,Herbert Burns,W,L,KO,7,1.0,7.0


# Network analysis


In [12]:
df[df.duplicated(subset=["losser", "winner"])]

,Date,winner,losser,BELT,BOUNES,Weight,TIME_FORMAT,Method,Method_detail,Judge1_score,...,ROUND,TIME,fighter1,fighter2,fighter1_result,fighter2_result,method,value,adj,adj_val
12848,2023-06-24,Justin Tafa,Austen Lane,NaN,NaN,Heavyweight,3,Could Not Continue,Eye injury,NaN,...,1,0:29,Austen Lane,Justin Tafa,NC,NC,other,0,1.0,0.0
19921,2023-03-04,Alexa Grasso,Valentina Shevchenko,belt,perf,Women'sFlyweight,5,Submission,Rear Naked Choke,NaN,...,4,4:34,Valentina Shevchenko,Alexa Grasso,L,W,sub,14,1.0,14.0
19964,2023-02-11,Islam Makhachev,Alexander Volkanovski,belt,fight,Lightweight,5,Decision - Unanimous,NaN,47 - 48.,...,5,5:00,Islam Makhachev,Alexander Volkanovski,W,L,Decision,10,1.0,10.0
19959,2023-02-11,Alonzo Menifield,Jimmy Crute,NaN,NaN,Light Heavyweight,3,Decision - Majority,NaN,27 - 29.,...,3,5:00,Jimmy Crute,Alonzo Menifield,D,D,Decision,0,1.0,0.0
19759,2022-08-20,Leon Edwards,Kamaru Usman,belt,perf,Welterweight,5,KO/TKO,Kick to Head At Distance,NaN,...,5,4:04,Kamaru Usman,Leon Edwards,L,W,KO,14,1.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29,1994-03-11,Patrick Smith,Johnny Rhodes,NaN,NaN,other,No,Submission,Guillotine Choke Standing,NaN,...,1,1:07,Patrick Smith,Johnny Rhodes,W,L,sub,7,-1.8,-12.6
28,1994-03-11,Patrick Smith,Scott Morris,NaN,NaN,other,No,KO/TKO,Elbows to Head From Mount,NaN,...,1,0:30,Patrick Smith,Scott Morris,W,L,KO,7,-1.8,-12.6
27,1994-03-11,Remco Pardoel,Alberta Cerra Leon,NaN,NaN,other,No,Submission,Other - Choke From Side Control Ezekiel Choke,NaN,...,1,9:51,Remco Pardoel,Alberta Cerra Leon,W,L,sub,7,-1.8,-12.6
26,1994-03-11,Johnny Rhodes,Fred Ettish,NaN,NaN,other,No,Submission,Other - Choke On Ground Bulldog Choke,NaN,...,1,3:07,Johnny Rhodes,Fred Ettish,W,L,sub,7,-1.8,-12.6


In [13]:
df.Date.max()

Timestamp('2024-03-16 00:00:00')

In [14]:
def calculate_rank(df):
    G_weighted = nx.from_pandas_edgelist(
        df, "losser", "winner", create_using=nx.DiGraph, edge_attr=["adj_val", "Date"]
    )

    weighted_pagerank = nx.pagerank(G_weighted, alpha=0.95)

    df_ranked = pd.DataFrame.from_dict(weighted_pagerank, orient="index")
    df_ranked.columns = ["page_rank"]
    df_ranked.sort_values(by=["page_rank"], ascending=False, inplace=True)
    df_ranked["run_date"] = df.Date.max()
    return df_ranked


df_ranked = calculate_rank(df)

In [15]:
df_ranked

,page_rank,run_date
Amanda Nunes,0.009480,2024-03-16
Israel Adesanya,0.008657,2024-03-16
Matt Hamill,0.008333,2024-03-16
Jon Jones,0.008069,2024-03-16
Stipe Miocic,0.008048,2024-03-16
...,...,...
Rodolfo Rubio Perez,0.000035,2024-03-16
Wagner Silva,0.000035,2024-03-16
Tim Gorman,0.000035,2024-03-16
Guto Inocente,0.000035,2024-03-16


In [16]:
def generate_ranks(df, file_name=None):

    weight_list = df["Weight"].unique()
    df_ranked = calculate_rank(df)
    df_w_rank_all = pd.DataFrame()

    for weight_to_show in weight_list:

        cond = (df["Weight"] == weight_to_show) & (
            df["Date"] >= dt - timedelta(weeks=52 * 3)
        )
        fighters_list = set(list(df[cond]["winner"]) + list(df[cond]["losser"]))
        df_w_rank = df_ranked[df_ranked.index.isin(fighters_list)].sort_values(
            by=["page_rank"], ascending=False
        )

        df_perf_select = (
            df_perf[
                (df_perf.index.isin(fighters_list))
                & (df_perf["weight"] == weight_to_show)
            ]
            .sort_values("perf_rat", ascending=False)[["perf_rat", "n_fights"]]
            .reset_index()
        )
        df_w_rank_select = df_w_rank.reset_index().rename({"index": "fighter"}, axis=1)

        df_w_rank_adj = pd.merge(
            df_w_rank_select, df_perf_select, on="fighter", how="inner"
        )  # .rename({'key_0':'fighter'}, axis=1)
        df_w_rank_adj["adj_p_rank"] = (
            df_w_rank_adj["page_rank"] * df_w_rank_adj["perf_rat"]
        )
        df_w_rank_adj.sort_values(
            by="adj_p_rank", ascending=False, inplace=True, ignore_index=True
        )

        df_w_rank_adj["weight"] = weight_to_show
        df_w_rank_adj["wight_class_rank"] = (
            df_w_rank_adj["adj_p_rank"]
            .rank(method="dense", ascending=False)
            .astype(int)
        )

        df_w_rank_all = pd.concat([df_w_rank_all, df_w_rank_adj])

    if file_name != "" and file_name is not None:
        df_w_rank_all[df_w_rank_all.wight_class_rank != "wight_class_rank"]
        df_w_rank_all.to_csv(file_name, mode="a", index=False, header=True)
        # df_w_rank_all.to_parquet(file_name, index=False, header=True)
    else:
        return df_w_rank_all


generate_ranks(df).head(20)

,fighter,page_rank,run_date,perf_rat,n_fights,adj_p_rank,weight,wight_class_rank
0,Israel Adesanya,0.008657,2024-03-16,0.750000,8.0,0.006492,Middleweight,1
1,Alex Pereira,0.005267,2024-03-16,0.857143,7.0,0.004515,Middleweight,2
2,Sean Strickland,0.004732,2024-03-16,0.750000,12.0,0.003549,Middleweight,3
3,Khamzat Chimaev,0.003258,2024-03-16,1.000000,1.0,0.003258,Middleweight,4
4,Dricus Du Plessis,0.002529,2024-03-16,1.000000,7.0,0.002529,Middleweight,5
5,Robbie Lawler,0.002155,2024-03-16,1.000000,2.0,0.002155,Middleweight,6
6,Robert Whittaker,0.003414,2024-03-16,0.625000,8.0,0.002134,Middleweight,7
7,Jared Cannonier,0.002423,2024-03-16,0.857143,7.0,0.002076,Middleweight,8
8,Paul Craig,0.002480,2024-03-16,0.500000,2.0,0.001240,Middleweight,9
9,Michel Pereira,0.001214,2024-03-16,1.000000,2.0,0.001214,Middleweight,10


In [17]:
LONG_FORMAT_DATA_START = "2020-01-01"
now = datetime.now()
formatted_date_time = now.strftime("%Y%m%d_%H%M")
print(formatted_date_time)
for dt in df["Date"].unique():
    if dt <= datetime.strptime(LONG_FORMAT_DATA_START, "%Y-%m-%d"):
        continue
    _df = df[df["Date"] <= dt]
    print(_df.shape, "     ", dt)
    df_w_rank_all = generate_ranks(
        _df, file_name=f"rank_long_{formatted_date_time}.csv"
    )

20240325_1914
(20431, 22)       2024-03-16 00:00:00
(20418, 22)       2024-03-09 00:00:00
(20404, 22)       2024-03-02 00:00:00
(20393, 22)       2024-02-24 00:00:00
(20381, 22)       2024-02-17 00:00:00
(20369, 22)       2024-02-10 00:00:00
(20355, 22)       2024-02-03 00:00:00
(20342, 22)       2024-01-20 00:00:00
(20330, 22)       2024-01-13 00:00:00
(20319, 22)       2023-12-16 00:00:00
(20307, 22)       2023-12-09 00:00:00
(20296, 22)       2023-12-02 00:00:00
(20284, 22)       2023-11-18 00:00:00
(20270, 22)       2023-11-11 00:00:00
(20256, 22)       2023-11-04 00:00:00
(20245, 22)       2023-10-21 00:00:00
(20232, 22)       2023-10-14 00:00:00
(20221, 22)       2023-10-07 00:00:00
(20211, 22)       2023-09-23 00:00:00
(20200, 22)       2023-09-16 00:00:00
(20188, 22)       2023-09-09 00:00:00
(20176, 22)       2023-09-02 00:00:00
(20165, 22)       2023-08-26 00:00:00
(20152, 22)       2023-08-19 00:00:00
(20140, 22)       2023-08-12 00:00:00
(20127, 22)       2023-08-05 00:00:0

In [18]:
df_long = pd.read_csv(f"rank_long_{formatted_date_time}.csv")
df_long = df_long[df_long.wight_class_rank != "wight_class_rank"]
numeric_columns = [
    "page_rank",
    "perf_rat",
    "n_fights",
    "adj_p_rank",
    "wight_class_rank",
]
df_long[numeric_columns] = df_long[numeric_columns].apply(pd.to_numeric)
df_long["run_date"] = pd.to_datetime(df_long["run_date"])  # .dt.date

df_long["run_date"].unique()

<DatetimeArray>
['2024-03-16 00:00:00', '2024-03-09 00:00:00', '2024-03-02 00:00:00',
 '2024-02-24 00:00:00', '2024-02-17 00:00:00', '2024-02-10 00:00:00',
 '2024-02-03 00:00:00', '2024-01-20 00:00:00', '2024-01-13 00:00:00',
 '2023-12-16 00:00:00',
 ...
 '2020-05-13 00:00:00', '2020-05-09 00:00:00', '2020-03-14 00:00:00',
 '2020-03-07 00:00:00', '2020-02-29 00:00:00', '2020-02-22 00:00:00',
 '2020-02-15 00:00:00', '2020-02-08 00:00:00', '2020-01-25 00:00:00',
 '2020-01-18 00:00:00']
Length: 178, dtype: datetime64[ns]

In [19]:
df_long.info()

<class 'pandas.core.frame.DataFrame'>
Index: 130455 entries, 0 to 130631
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   fighter           130455 non-null  object        
 1   page_rank         130455 non-null  float64       
 2   run_date          130455 non-null  datetime64[ns]
 3   perf_rat          130455 non-null  float64       
 4   n_fights          130455 non-null  float64       
 5   adj_p_rank        130455 non-null  float64       
 6   weight            130455 non-null  object        
 7   wight_class_rank  130455 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(1), object(2)
memory usage: 9.0+ MB


In [20]:
df_long[df_long["fighter"].str.contains("Islam")]

,fighter,page_rank,run_date,perf_rat,n_fights,adj_p_rank,weight,wight_class_rank
355,Islam Makhachev,0.004331,2024-03-16,1.0,7.0,0.004331,Lightweight,1
1037,Islam Makhachev,0.004331,2024-03-16,1.0,2.0,0.004331,other,1
1102,Islam Makhachev,0.004337,2024-03-09,1.0,7.0,0.004337,Lightweight,1
2135,Islam Makhachev,0.004337,2024-03-09,1.0,2.0,0.004337,other,1
2309,Islam Makhachev,0.004366,2024-03-02,1.0,7.0,0.004366,Lightweight,1
...,...,...,...,...,...,...,...,...
128681,Islam Makhachev,0.001257,2020-02-22,1.0,7.0,0.001257,Lightweight,5
128955,Islam Makhachev,0.001262,2020-02-15,1.0,7.0,0.001262,Lightweight,5
129739,Islam Makhachev,0.001263,2020-02-08,1.0,7.0,0.001263,Lightweight,5
130158,Islam Makhachev,0.001288,2020-01-25,1.0,7.0,0.001288,Lightweight,5


In [26]:
_df = df_long[
    (df_long["weight"].str.contains("Lightweight")) & (df_long["wight_class_rank"] < 10)
].sort_values(["wight_class_rank", "run_date"])
_df.head()

,fighter,page_rank,run_date,perf_rat,n_fights,adj_p_rank,weight,wight_class_rank
130420,Michael Johnson,0.005937,2020-01-18,0.6,5.0,0.003562,Lightweight,1
130154,Michael Johnson,0.005929,2020-01-25,0.6,5.0,0.003558,Lightweight,1
129735,Michael Johnson,0.005803,2020-02-08,0.6,5.0,0.003482,Lightweight,1
128951,Michael Johnson,0.005767,2020-02-15,0.6,5.0,0.003460,Lightweight,1
128677,Michael Johnson,0.005705,2020-02-22,0.6,5.0,0.003423,Lightweight,1


In [24]:
# df_w_rank_all[(df_w_rank_all['n_fights']>1)].to_csv('w_ranking.csv')
# df_w_rank_all.to_csv("w_ranking.csv")

In [25]:
# ['Welterweight', 'Middleweight', 'Featherweight', 'Bantamweight',
#        "Women's Bantamweight", "Women'sFlyweight", 'Flyweight',
#        'Heavyweight', 'Lightweight', 'other', "Women's Strawweight",
#        'Light Heavyweight', "Women's Featherweight"]

In [ ]:
# df_w_rank_all[
#     (df_w_rank_all["n_fights"] > 3) & (df_w_rank_all["weight"] == "Featherweight")
# ].reset_index().head(25)

,index,fighter,page_rank,perf_rat,n_fights,adj_p_rank,weight
0,0,Max Holloway,0.004372,0.800000,5.0,0.003497,Featherweight
1,1,Ilia Topuria,0.003023,1.000000,5.0,0.003023,Featherweight
2,2,Alexander Volkanovski,0.003328,0.857143,7.0,0.002853,Featherweight
3,5,Movsar Evloev,0.001297,1.000000,5.0,0.001297,Featherweight
4,6,Edson Barboza,0.002323,0.500000,8.0,0.001161,Featherweight
5,7,Josh Emmett,0.001630,0.666667,6.0,0.001086,Featherweight
6,8,Cub Swanson,0.001973,0.500000,6.0,0.000986,Featherweight
7,9,Lerone Murphy,0.000908,1.000000,4.0,0.000908,Featherweight
8,10,Darren Elkins,0.001293,0.625000,8.0,0.000808,Featherweight
9,11,Calvin Kattar,0.001470,0.500000,4.0,0.000735,Featherweight


In [27]:
def pfp_performance(m):
    df_1 = (
        df[(df["Date"] >= dt - timedelta(weeks=52 * m))]
        .groupby(["fighter1", "fighter1_result"])["Weight"]
        .count()
        .unstack(level=-1)
        .fillna(0)
    )
    df_1.index.set_names(["fighter"], inplace=True)
    df_2 = (
        df[(df["Date"] >= dt - timedelta(weeks=52 * m))]
        .groupby(["fighter2", "fighter2_result"])["Weight"]
        .count()
        .unstack(level=-1)
        .fillna(0)
    )
    df_2.index.set_names(["fighter"], inplace=True)
    df_perf = df_1.add(df_2, fill_value=0)
    df_perf["n_fights"] = df_perf.sum(axis=1)
    df_perf["perf_rat"] = (df_perf["n_fights"] - df_perf["L"]) / df_perf["n_fights"]
    df_perf.columns = df_perf.columns.values
    df_perf = df_perf.sort_values(
        by=["perf_rat", "n_fights"], ascending=False
    ).reset_index(
        level=-1
    )  # [df_perf.n_fights>2]
    return df_perf

In [28]:
df_perf_pfp = pfp_performance(3)
df_pfp = pd.merge(
    df_ranked.reset_index().rename({"index": "fighter"}, axis=1),
    df_perf_pfp,
    on="fighter",
    how="inner",
)
df_pfp["adj_p_rank"] = df_pfp["page_rank"] * df_pfp["perf_rat"]
df_pfp.sort_values(by="adj_p_rank", ascending=False, inplace=True, ignore_index=True)
# df_pfp[(df_pfp['n_fights']>1)].to_csv('pfp_ranking.csv')
df_pfp.to_csv("pfp_ranking.csv")
df_pfp[(df_pfp["n_fights"] > 7)].head(25)

,fighter,page_rank,run_date,D,L,NC,W,n_fights,perf_rat,adj_p_rank
0,Amanda Nunes,0.009480,2024-03-16,0.0,5.0,0.0,44.0,49.0,0.897959,0.008513
1,Jon Jones,0.008069,2024-03-16,0.0,3.0,3.0,61.0,67.0,0.955224,0.007708
2,Israel Adesanya,0.008657,2024-03-16,0.0,5.0,0.0,33.0,38.0,0.868421,0.007518
3,Francis Ngannou,0.007647,2024-03-16,0.0,6.0,0.0,35.0,41.0,0.853659,0.006528
4,Stipe Miocic,0.008048,2024-03-16,0.0,12.0,0.0,42.0,54.0,0.777778,0.006259
5,Matt Hamill,0.008333,2024-03-16,0.0,15.0,0.0,30.0,45.0,0.666667,0.005555
6,Derrick Lewis,0.007673,2024-03-16,0.0,22.0,0.0,51.0,73.0,0.698630,0.005360
7,Curtis Blaydes,0.006719,2024-03-16,0.0,10.0,3.0,33.0,46.0,0.782609,0.005259
8,Alex Pereira,0.005267,2024-03-16,0.0,1.0,0.0,8.0,9.0,0.888889,0.004682
9,Kamaru Usman,0.004984,2024-03-16,0.0,3.0,0.0,44.0,47.0,0.936170,0.004666
